### Computing the error of the numerical discretization with respect to reference solution on a very fine grid
#### Used for Figures 7 and 9

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from clawpack import pyclaw
from piston_Lagrangian import setup

In [ ]:
piston_freq = 1
p_domain = 30
p = 20
#Reference solution
xmax1 = p_domain*2*np.pi 
N = 250*12 #Cells per period at final time
M = 0.4
mx1 = p_domain*N #Number of grid points
tfinal = p*2*np.pi
print("Reference solution will have {} cells".format(mx1))

##### Uncomment the following cell to generate a reference numerical solution with a very fine grid (expensive)

In [ ]:
# #Run reference solution
# piston_problem1 = setup(outdir="./data_convergence",tfinal=tfinal, xmax=xmax1, mx=mx1, M=M, CFL=0.8,limiting=1,solver_type='sharpclaw',
# order=2, time_integrator='Heun', nout=100, piston_freq=piston_freq)
# # piston_problem1.verbosity = 0
# piston_problem1.run()
# piston_problem1.solver.__del__()

### Error Measurement

In [ ]:
def relative_L1_error(u1,u2):
    #u1 will be a reference solution
    #u2 will be a 'numerical' solution
    return np.sum(np.abs(u1-u2))/np.sum(np.abs(u2))

def max_error_discretization(piston_problem1,xlimRHS):
    xcoarse = piston_problem1.grid.x.centers
    indx = np.where(xcoarse<=xlimRHS) #Avoiding noise from the "Sponge layer" region 

    u_init_num = piston_problem1.frames[0].q[1]
    ref_init_sol = pyclaw.Solution()
    ref_init_sol.read(path="./data_convergence", file_format='ascii', file_prefix='fort', frame=0)
    xfine = ref_init_sol.grid.x.centers
    indx2 = get_nearest_idxs(xcoarse=xcoarse, xfine=xfine) #To get relevant points from the reference solution

    max_error = 0

    for i in range(len(piston_problem1.frames)):
        u_num = piston_problem1.frames[i].q[1]
        u_num = u_num[indx]
        u_ref = get_coarse_from_reference(xcoarse = xcoarse, frame=i,idxs = indx2)
        u_ref = u_ref[indx]
        local_error = relative_L1_error(u_ref,u_num)
        if local_error > max_error:
            max_error = local_error
    return max_error

def get_nearest_idxs(xcoarse, xfine):
    indx = []
    for x in xcoarse:
        diff = np.abs(xfine-x)
        indx.append(np.argmin(diff))
    return indx


def get_coarse_from_reference(xcoarse, frame=100,idxs=None):
    sol = pyclaw.Solution()
    sol.read(path="./data_convergence", file_format='ascii', file_prefix='fort', frame=frame)
    ufine = sol.q[1]
    if idxs is None:
        xfine = sol.grid.x.centers
        idxs = get_nearest_idxs(xcoarse, xfine)
    ucoarse = ufine[idxs]
    return ucoarse


In [ ]:
N_grids = [10.,50.,250.]  #Cells per period at final time
errors = []

#Fixed parameters
piston_freq = 1
p_domain = 30
p = 20
xmax1 = p_domain*2*np.pi 
xs = p*2*np.pi #Begining of sponge layer
M = 0.4
tfinal = p*2*np.pi
for N in N_grids:
    mx1 = p_domain*N #Number of grid points
    piston_problem1 = setup(outdir="./_output",tfinal=tfinal, xmax=xmax1, mx=mx1, M=M, CFL=0.8,limiting=1,solver_type='sharpclaw',
    order=2, time_integrator='Heun', nout=100, piston_freq=piston_freq)
    piston_problem1.verbosity = 0
    piston_problem1.run()
    piston_problem1.solver.__del__()
    errors.append(max_error_discretization(piston_problem1,xlimRHS=xs))
    

In [ ]:
errors